# NYC Flights Database

In [1]:
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [2]:
df = pd.read_csv(r"C:\Users\saads\OneDrive\Documents\NYC.csv")
df = df.drop(df.columns[[0]], axis=1)

In [3]:
df.sample(3)

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
307789,2013,8,30,857.0,-4.0,1151.0,-13.0,B6,N637JB,63,JFK,SEA,325.0,2422,8.0,57.0
25480,2013,1,30,1117.0,27.0,1358.0,-7.0,B6,N584JB,373,LGA,FLL,145.0,1076,11.0,17.0
205054,2013,5,13,1528.0,-1.0,1713.0,-40.0,UA,N77530,1101,EWR,DEN,207.0,1605,15.0,28.0


In [4]:
df['carrier'].unique()


array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

In [5]:
month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)


# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

In [6]:
def validate():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(df['month'] == month.value, df['carrier'] == textbox.value,
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]

        else:
            filter_list = [i and j for i, j in
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'


origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

In [7]:
container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])


In [8]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd

# Load the CSV file into a DataFrame
file_path = r"C:\Users\saads\OneDrive\Documents\NYC.csv"
df = pd.read_csv(file_path)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the web app
app.layout = html.Div([
    # Define an input component (e.g., a dropdown)
    dcc.Dropdown(
        id='your-input-component',  # Update 'your-input-component'
        options=[
            {'label': 'Option 1', 'value': 'option1'},
            {'label': 'Option 2', 'value': 'option2'}
        ],
        value='option1'  # Set the default value
    ),
    dcc.Graph(id='scatter-plot'),
])

# Define a callback to update the scatter plot
@app.callback(
    Output('scatter-plot', 'figure'),
    Input('your-input-component', 'value')  # Match the input component id
)
def update_scatter_plot(selected_value):
    # Perform any data filtering or processing here based on user input
    # For example, you can use df to create a scatter plot
    # and update the figure based on the selected_value

    # Return a plotly figure
    return {
        'data': [
            # Define your scatter plot data here
        ],
        'layout': {
            'title': 'Scatter Plot',
            # Customize the layout of your plot
        }
    }

if __name__ == '__main__':
    app.run_server(debug=True, port=8054)


C:\Users\saads\AppData\Local\Temp\ipykernel_4616\1772005961.py:2: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

C:\Users\saads\AppData\Local\Temp\ipykernel_4616\1772005961.py:3: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [9]:
# Import necessary libraries
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load the flight data
flight_data = pd.read_csv(r"C:\Users\saads\OneDrive\Documents\NYC.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Flight Data Analysis Dashboard"),
    
    # Dropdown for selecting a carrier
    dcc.Dropdown(
        id='carrier-dropdown',
        options=[{'label': carrier, 'value': carrier} for carrier in flight_data['carrier'].unique()],
        value=flight_data['carrier'].unique()[0],
        multi=False
    ),
    
    # Scatter plot for Departure Delay vs Arrival Delay
    dcc.Graph(id='scatter-plot'),
    
    # Bar chart for Departure Delay by Hour
    dcc.Graph(id='bar-chart'),
])

# Define callback functions to update plots based on user inputs
@app.callback(
    Output('scatter-plot', 'figure'),
    Output('bar-chart', 'figure'),
    Input('carrier-dropdown', 'value')
)
def update_plots(selected_carrier):
    filtered_data = flight_data[flight_data['carrier'] == selected_carrier]
    
    # Scatter plot
    scatter_plot = px.scatter(filtered_data, x='dep_delay', y='arr_delay', title='Departure Delay vs Arrival Delay')
    
    # Bar chart
    bar_chart = px.bar(filtered_data, x='hour', y='dep_delay', title='Departure Delay by Hour')
    
    return scatter_plot, bar_chart

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port=8061)
